In [0]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from networkx.algorithms.components.connected import connected_components
import warnings
import networkx
import numpy as np
import json
import nltk
from nltk.corpus import stopwords
import pprint

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
df = pd.read_csv('/content/drive/My Drive/small_kaggle_clean.csv')
print(df.shape)
df['sst3']

(206, 7)


0      CCEEECCCCCCCCCC
1      CCEEECCCCCCCCCC
2      CCEEECCCCCCCCCC
3      CCCCCCCCCCCCCCC
4      CCCCCCCCCCCCCCC
            ...       
201    CCCCCCCCCCCCCCC
202    CCCCCCCCCCCCCCC
203    CCCCCCCCCECCCCC
204    CCCCCCCCCCCCCCC
205    CCCCCCCCCCCCCCC
Name: sst3, Length: 206, dtype: object

In [0]:
def preprocess(df):

    print("Original: ",df.shape)
    for index, row in df.iterrows():
        if index < 3500:
        # print(index)
            if len(row['seq']) % 3 != 0 or '*' in row['seq']:
                df.drop(index, inplace=True)
        else:
            break

    print("New: " ,df.shape)
    print(df['seq'].head(10))
    
    return df['seq']

In [0]:
def process_n_dump_to_csv(data):
    df = preprocess(data)
    print("Size of dataframe: ", len(df))
    df.to_csv("sentences.csv")
    return df


In [9]:
wordmodelfile ='/content/drive/My Drive/protVec_clean.txt'
wordmodel = KeyedVectors.load_word2vec_format(wordmodelfile, binary = False, limit=200000)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def seq_grams(s1):
    '''
    i/p:  protein sequence
    o/p:  list where each element has 3 chars 
    '''
    s1_words=[]
    for count in range(0,len(s1)-2):
        s1_words.append(s1[count:count+3])
        count+=3
    return s1_words

def similarityIndex(s1, s2, wordmodel):
    '''
    To compare two protein sequences and return their similarity score based on the gensim model
    '''
    if s1 == s2:
        return 1.0
    
    s1words = seq_grams(s1)
    s2words = seq_grams(s2)

    s1words = list(s1words)
    s2words = list(s2words)    

    s1set = set(s1words)
    s2set = set(s2words)

    vocab = wordmodel.vocab
    
    if len(s1set & s2set) == 0:
        return 0.0
    for word in s1set.copy():
        if (word not in vocab):
            s1words.remove(word)
    for word in s2set.copy():
        if (word not in vocab):
            s2words.remove(word)
    try:
        s = wordmodel.n_similarity(s1words, s2words)
    except:
        s = 0
    return s


In [0]:
n = df.shape[0]
similarity_matrix = [[-1 for c in range(n)] for r in range(n)]

In [12]:
row = 0
for seq_1 in df['seq']:
    column = 0
    for seq_2 in df['seq']:
        if seq_1 == seq_2:
            column += 1
            continue
        similarity_matrix[row][column] = similarityIndex(seq_1, seq_2, wordmodel)
        column += 1
    print("Row: ",row)
    row += 1


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Row:  0
Row:  1
Row:  2
Row:  3
Row:  4
Row:  5
Row:  6
Row:  7
Row:  8
Row:  9
Row:  10
Row:  11
Row:  12
Row:  13
Row:  14
Row:  15
Row:  16
Row:  17
Row:  18
Row:  19
Row:  20
Row:  21
Row:  22
Row:  23
Row:  24
Row:  25
Row:  26
Row:  27
Row:  28
Row:  29
Row:  30
Row:  31
Row:  32
Row:  33
Row:  34
Row:  35
Row:  36
Row:  37
Row:  38
Row:  39
Row:  40
Row:  41
Row:  42
Row:  43
Row:  44
Row:  45
Row:  46
Row:  47
Row:  48
Row:  49
Row:  50
Row:  51
Row:  52
Row:  53
Row:  54
Row:  55
Row:  56
Row:  57
Row:  58
Row:  59
Row:  60
Row:  61
Row:  62
Row:  63
Row:  64
Row:  65
Row:  66
Row:  67
Row:  68
Row:  69
Row:  70
Row:  71
Row:  72
Row:  73
Row:  74
Row:  75
Row:  76
Row:  77
Row:  78
Row:  79
Row:  80
Row:  81
Row:  82
Row:  83
Row:  84
Row:  85
Row:  86
Row:  87
Row:  88
Row:  89
Row:  90
Row:  91
Row:  92
Row:  93
Row:  94
Row:  95
Row:  96
Row:  97
Row:  98
Row:  99
Row:  100
Row:  101
Row:  102
Row:  103
Row:  104
Row:  105
Row:  106
Row:  107
Row:  108
Row:  109
Row:  110


In [0]:
setlist = []
index = 0
for score_row, response in zip(similarity_matrix, df['seq']):
    max_sim_index = index
    if np.array(score_row).sum() > 0:
        max_sim_index = np.array(score_row).argmax()
    if set([response, df['seq'][max_sim_index]]) not in setlist:
        setlist.append([response, df['seq'][max_sim_index]])
    index += 1


In [0]:

def toGraph(l):
    '''
    It takes in a list of lists and returns a graph object, 
    assigning nodes and edges from each sub-list object
    '''
    G = networkx.Graph()
    for part in l:
        G.add_nodes_from(part)
        G.add_edges_from(toEdges(part))
    return G

def toEdges(l):
    '''
    It treats args(1) 'l' as a graph and returns (implicitly) it's edges 
    '''
    it = iter(l)
    last = next(it)

    for current in it:
        yield last, current
        last = current 

In [15]:
G = toGraph(setlist)
setlist = list(connected_components(G))
print(setlist)

[{'ARTKQTARKSTGGKA'}, {'EPVDPKLEPWKHPGS'}, {'VQYLTRSAIRRASTI', 'STASTVEYSTVVHSG', 'KSAVTTVVNPKYEGK'}, {'KETAAAKFERQHMDS', 'KETAAACFERCHMDS', 'GELIGILNAAKVPAD'}, {'KMRMATPLLMQALPM', 'PVSKMRMATPLLMQA'}, {'ELNPNAEVWGAPVLH', 'HENIDWGEPKDKEVW'}, {'RKRTYETFKSIMKKS', 'LSRVKSLKKSLRQSF'}, {'PVLFPGQPFGQPPLG', 'SPGRLFPGAPPTAKK', 'YEPSTTARKVGRPGR'}, {'AVPIAQKSEPHSLSS', 'PQAQQKSLLQQLLTE'}, {'RTFRQVQSSISDFYD'}, {'RKTVAKPKGPSGSPW', 'RHKILHRLLQEGSPS', 'LTERHKILHRLLQEG'}, {'LPQDRGFLVVQGDPR'}, {'ECCHRQLLCCLRFV*'}, {'KKESRVVVWRLPPLH', 'APRRLPIFNRISVSE'}, {'ASEDELVAEFLQDQN'}, {'NTGRTEAWKVLSPQG', 'SPQGGGPWDSVARVL', 'DGTCVAARTRPVLSY'}, {'RRRPRPPYLPRPRPP', 'DLSPRPSPNPHPVSQ', 'GGRRRRRTWYFGGGK'}, {'DFSIVGSLPRDFELS', 'SLMRVQAHIRKRMVA', 'GSLLRVQAHIRKKMV'}, {'SQETFSDLWKLLPEN'}, {'QKVTFPKMKIPKFTF'}, {'TTGEIIGDIRQAHCN'}, {'ATEYRVRVNSAYQDK'}, {'NLGLEDIIRKALMGS'}, {'SAPATGGVKKPHRYR', 'APATGGVKKPHRYRP'}, {'KMLEGQNAHFRYKNR'}, {'FLNKDLEVDGHFVTM'}, {'ADSTITIRGYVRDNG'}, {'DGVFTTPCDPEYAGG'}, {'SKQFTSSTSYNRGDS'}, {'SIDMVDSP

In [16]:
novel_sub_categories = {}
index = 0
for category in setlist:
    novel_sub_categories[index] = list(category)
    index += 1

result = novel_sub_categories
pprint.pprint(result)

{0: ['ARTKQTARKSTGGKA'],
 1: ['EPVDPKLEPWKHPGS'],
 2: ['VQYLTRSAIRRASTI', 'STASTVEYSTVVHSG', 'KSAVTTVVNPKYEGK'],
 3: ['KETAAAKFERQHMDS', 'KETAAACFERCHMDS', 'GELIGILNAAKVPAD'],
 4: ['KMRMATPLLMQALPM', 'PVSKMRMATPLLMQA'],
 5: ['ELNPNAEVWGAPVLH', 'HENIDWGEPKDKEVW'],
 6: ['RKRTYETFKSIMKKS', 'LSRVKSLKKSLRQSF'],
 7: ['PVLFPGQPFGQPPLG', 'SPGRLFPGAPPTAKK', 'YEPSTTARKVGRPGR'],
 8: ['AVPIAQKSEPHSLSS', 'PQAQQKSLLQQLLTE'],
 9: ['RTFRQVQSSISDFYD'],
 10: ['RKTVAKPKGPSGSPW', 'RHKILHRLLQEGSPS', 'LTERHKILHRLLQEG'],
 11: ['LPQDRGFLVVQGDPR'],
 12: ['ECCHRQLLCCLRFV*'],
 13: ['KKESRVVVWRLPPLH', 'APRRLPIFNRISVSE'],
 14: ['ASEDELVAEFLQDQN'],
 15: ['NTGRTEAWKVLSPQG', 'SPQGGGPWDSVARVL', 'DGTCVAARTRPVLSY'],
 16: ['RRRPRPPYLPRPRPP', 'DLSPRPSPNPHPVSQ', 'GGRRRRRTWYFGGGK'],
 17: ['DFSIVGSLPRDFELS', 'SLMRVQAHIRKRMVA', 'GSLLRVQAHIRKKMV'],
 18: ['SQETFSDLWKLLPEN'],
 19: ['QKVTFPKMKIPKFTF'],
 20: ['TTGEIIGDIRQAHCN'],
 21: ['ATEYRVRVNSAYQDK'],
 22: ['NLGLEDIIRKALMGS'],
 23: ['SAPATGGVKKPHRYR', 'APATGGVKKPHRYRP'],
 24: [

In [0]:
with open('clusters.json', 'w') as temp:
    json.dump(result, temp)